In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
dmeeting = pd.read_parquet('./data/data_meeting.parquet')
dagenda = pd.read_parquet('./data/data_agenda.parquet')
data_speech1 = pd.read_parquet('./data/data_speech1.parquet')
data_speech2 = pd.read_parquet('./data/data_speech2.parquet')
data_speech3 = pd.read_parquet('./data/data_speech3.parquet')
parMem = pd.read_parquet('./data/parliament_members.parquet')

dspeech = pd.concat([data_speech1, data_speech2, data_speech3], axis=0)

## Andreas

## Eisuke

## Anders

## Relative Party

In [ ]:
dspeech = pd.merge(dspeech, dmeeting[['meeting_id', 'date']], on = 'meeting_id')
dspeech['year'] = dspeech['date'].dt.to_period('Y')
dspeech = dspeech[dspeech['speaker_party'].isin(['S', 'DF', 'ALT', 'DD', 'EL', 'FG', 'KD', 'KF', 'LA', 'M', 'NB', 'RV', 'S', 'SF', 'V'])]

In [ ]:
weekly_ag = pd.DataFrame(columns=['date', 'speaker_party', 'percent_C', 'MA52'])

for party in dspeech['speaker_party'].unique().tolist():
    weekly = dspeech[dspeech['speaker_party']==party].groupby(dspeech['date'].dt.to_period('M')).agg(
        count_C=('label', lambda x: (x == 'C').sum()),
        count_NC=('label', lambda x: (x == 'NC').sum())
    )
    weekly['percent_C'] = (weekly['count_C'] / (weekly['count_C'] + weekly['count_NC'])) * 100
    weekly = weekly.reset_index()
    weekly['date'] = weekly['date'].dt.to_timestamp()
    weekly['MA52'] = weekly['percent_C'].rolling(window=48, center=True, min_periods=1).mean()
    weekly['speaker_party'] = party
    weekly_ag = pd.concat([weekly_ag, weekly])

first_MA52 = weekly_ag.groupby('speaker_party')['MA52'].transform('first')    
mean_MA52 = weekly_ag.groupby('speaker_party')['MA52'].transform('mean')
weekly_ag['percent_C_relative'] = (weekly_ag['MA52'] / first_MA52 -1) * 100
weekly_ag['percent_C_relative_all'] = (weekly_ag['MA52'] / mean_MA52 -1 ) * 100


# def mean_of_first_three(series):
#     return series.head(12).mean()

# mean_first_three_MA52 = weekly_ag.groupby('speaker_party')['MA52'].transform(mean_of_first_three)
# weekly_ag['mean_first_three_MA52'] = mean_first_three_MA52
# weekly_ag['percent_C_relative_first_threeM'] = (weekly_ag['MA52'] / weekly_ag['mean_first_three_MA52'] -1) * 100

In [ ]:
#Colors from here https://gist.github.com/josiahayres-ibm/9ddc2413ee61b56f915ba602f2b9305d
carbon_categorical_light = ["#6929c4", "#1192e8", "#005d5d","#9f1853","#fa4d56","#570408","#198038","#002d9c","#ee538b","#b28600","#009d9a","#012749","#8a3800","#a56eff"]
sns.set_palette(carbon_categorical_light)
sns.palplot(sns.color_palette())

In [ ]:
weekly_ag = weekly_ag[~weekly_ag['speaker_party'].isin(['M', 'KD', 'DD', 'FG'])]

from matplotlib.dates import YearLocator, DateFormatter
from matplotlib.ticker import MultipleLocator, FuncFormatter
plt.figure(figsize=(12, 12))
sns.lineplot(x='date', y='percent_C_relative', hue='speaker_party', marker=None,  data=weekly_ag, legend=False, palette=carbon_categorical_light)
plt.axhline(y=0, color='black', linestyle='--', linewidth=1)
plt.ylabel('% Climate related speech items relative to baseline', fontsize=16)
plt.xlabel(None) 

ax = plt.gca()
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter('%Y'))
plt.xticks(rotation=45, ha='center')

def percentage_formatter(x, pos):
    return f"{x:.0f}%"

ax.yaxis.set_major_locator(MultipleLocator(20))
ax.yaxis.set_major_formatter(FuncFormatter(percentage_formatter))
ax.tick_params(axis='both', which='major', labelsize=14) 
plt.tight_layout()
for party in weekly_ag['speaker_party'].unique():
    last_data_point = weekly_ag.loc[weekly_ag['speaker_party'] == party].iloc[-1]
    plt.annotate(party, (last_data_point['date'], last_data_point['percent_C_relative']),
                 textcoords="offset points", xytext=(25,0), ha='right', fontsize=14, color='black')
#plt.savefig('./figures/parties_change_final.svg', format='svg')

## Overall party

In [ ]:
party_name_mapping = {
    'FG':'Frie Grønne',
    'ALT':'Alternativet',
    'M':'Moderaterne',
    'SF':'Socialistisk Folkeparti',
    'EL':'Enhedslisten',
    'RV':'Radikale Venstre',
    'V':'Venstre',
    'S':'Socialdemokratiet',
    'DD':'Danmarks Demokratterne',
    'KF':'Konservative Folkeparti',
    'LA':'Liberal Alliance',
    'NB':'Nye Borgerlige',
    'DF':'Dansk Folkeparti',
    'KD':'Kristendemokraterne',  
}

party = dspeech.groupby(['speaker_party']).agg(
    count_C=('label', lambda x: (x == 'C').sum()),
    count_NC=('label', lambda x: (x == 'NC').sum())
)
party['percent_C'] = (party['count_C'] / (party['count_C']+party['count_NC']))*100
party['percent_C_mean_centered'] = party['percent_C'] - party['percent_C'].mean()
#party['colors'] = [sns.xkcd_rgb['medium green'] if val >= 0 else sns.xkcd_rgb['amber'] for val in party['percent_C_mean_centered']]

party = party.reset_index()
party['speaker_party_new'] = party['speaker_party'].replace(party_name_mapping)

party['rank'] = party['percent_C_mean_centered'].rank(ascending=False)

# Sort the DataFrame by rank
party_sorted = party.sort_values('rank')
# Set the color based on the sign of percent_C_mean_centered
colors = [sns.xkcd_rgb['medium green'] if val >= 0 else sns.xkcd_rgb['light red'] for val in party_sorted['percent_C_mean_centered']]
plt.figure(figsize=(13, 7))
sns.barplot(x='percent_C_mean_centered', y='speaker_party_new', palette=colors, data=party_sorted, orient='h')
plt.xlabel('Average % climate related speech items', fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylabel('')
#sns.despine(left=True, top=True, right=True, bottom=True)

for i, (val, color, party) in enumerate(zip(party_sorted['percent_C_mean_centered'], colors, party_sorted['speaker_party_new'])):
    if party in ['Enhedslisten', 'Radikale Venstre', 'Venstre']:
        ha_value = 'right'
    else:
        ha_value = 'right' if color == sns.xkcd_rgb['medium green'] else 'left'
    plt.text(val, i, f'{val:.2f}%', va='center', ha=ha_value, fontsize=14, color='black')
    
plt.tight_layout()
#plt.savefig('./figures/overall_party.svg', format='svg')